In [1]:
import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('./datasets/spam.csv', encoding='ISO-8859-1')

In [3]:
le = LabelEncoder()

In [4]:
# same as df.values
data = df.to_numpy()

In [5]:
data

array([['ham',
        'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
        nan, nan, nan],
       ['ham', 'Ok lar... Joking wif u oni...', nan, nan, nan],
       ['spam',
        "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
        nan, nan, nan],
       ...,
       ['ham',
        'Pity, * was in mood for that. So...any other suggestions?', nan,
        nan, nan],
       ['ham',
        "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free",
        nan, nan, nan],
       ['ham', 'Rofl. Its true to its name', nan, nan, nan]], dtype=object)

In [6]:
# We have 5572 emails
len(data)

5572

In [7]:
# Not spam (ham)
data[0]

array(['ham',
       'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
       nan, nan, nan], dtype=object)

In [8]:
# spam
data[2]

array(['spam',
       "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
       nan, nan, nan], dtype=object)

In [9]:
# From above two examples, 0th column is the y or answer
# Note: Only[:,1] and not[:,1:] as only two columns
X = data[:,1]
y = data[:,0] 

In [10]:
X.shape, y.shape

((5572,), (5572,))

In [11]:
tokenizer = RegexpTokenizer('\w+')
sw = set(stopwords.words('english'))
ps = PorterStemmer()

In [12]:
def getStem(review):
    # Step 1: Convert to lowercase
    review = review.lower()
    # Step 2: Break down into words
    tokens = tokenizer.tokenize(review) # Breaking into small words
    # Step 3: Remove the stopwords
    removed_stopwords = [w for w in tokens if w not in sw]
    # Step 4: Stem the words
    stemmed_words = [ps.stem(token) for token in removed_stopwords]
    # Rejoin the words to remake the sentence
    clean_review = ' '.join(stemmed_words)
    return clean_review

In [13]:
# Get a clean document
def getDoc(document):
    d = []
    for doc in document:
        d.append(getStem(doc))
    return d

In [14]:
stemmed_doc = getDoc(X)

In [15]:
stemmed_doc[:10]

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri 2 wkli comp win fa cup final tkt 21st may 2005 text fa 87121 receiv entri question std txt rate c appli 08452810075over18',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl 3 week word back like fun still tb ok xxx std chg send å 1 50 rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press 9 copi friend callertun',
 'winner valu network custom select receivea å 900 prize reward claim call 09061701461 claim code kl341 valid 12 hour',
 'mobil 11 month u r entitl updat latest colour mobil camera free call mobil updat co free 08002986030']

In [16]:
cv = CountVectorizer()

In [17]:
# Create vocab
vc = cv.fit_transform(stemmed_doc)

In [18]:
X = vc.todense() # OR vc.toarray()

In [19]:
 X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

In [20]:
from sklearn.naive_bayes import MultinomialNB

In [21]:
model = MultinomialNB()

In [22]:
model.fit(X_train,y_train)

MultinomialNB()

In [23]:
model.score(X_test,y_test)

0.977705274605764

In [24]:
messages = [
    """
    Hi Kunal,
We invite you to participate in MishMash - India’s largest online diversity hackathon. 
The hackathon is a Skillenza initiative and sponsored by Microsoft, Unity, Unilever, Gojek, Rocketium and Jharkhand Government. 
We have a special theme for you - Deep Tech/Machine Learning - sponsored by Unilever, which will be perfect for you.
    """,
    """Join us today at 12:00 PM ET / 16:00 UTC for a Red Hat DevNation tech talk on AWS Lambda and serverless Java with Bill Burke.
Have you ever tried Java on AWS Lambda but found that the cold-start latency and memory usage were far too high? 
In this session, we will show how we optimized Java for serverless applications by leveraging GraalVM with Quarkus to 
provide both supersonic startup speed and a subatomic memory footprint.""",

    """We really appreciate your interest and wanted to let you know that we have received your application.
There is strong competition for jobs at Intel, and we receive many applications. As a result, it may take some time to get back to you.
Whether or not this position ends up being a fit, we will keep your information per data retention policies, 
so we can contact you for other positions that align to your experience and skill set.
"""
]

In [25]:
def prepare(messages):
    d = getDoc(messages)
    # dont do fit_transform!! it will create new vocab.
    return cv.transform(d)

messages = prepare(messages)

In [27]:
messages

<3x7213 sparse matrix of type '<class 'numpy.int64'>'
	with 72 stored elements in Compressed Sparse Row format>

In [30]:
messages.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [26]:
y_pred = model.predict(messages)
y_pred

array(['ham', 'spam', 'ham'], dtype='<U4')